In [1]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from joblib import Parallel, delayed
import matplotlib.pyplot as plt

Using Theano backend.


In [2]:
#Loading the training and testing data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [3]:
#Seed for reproducibilty
np.random.seed(1338)

#Selecting 6000 random examples from the test data
test_rows = np.random.randint(0,X_test.shape[0],6000)
X_test = X_test[test_rows]
Y = y_test[test_rows]
#Converting the output to binary classification(Six=1,Not Six=0)
Y_test = Y == 6
Y_test = Y_test.astype(int)

#Selecting the 5918 examples where the output is 6
X_six = X_train[y_train == 6]
Y_six = y_train[y_train == 6]
#Selecting the examples where the output is not 6
X_not_six = X_train[y_train != 6]
Y_not_six = y_train[y_train != 6]

#Selecting 6000 random examples from the data that contains only the data where the output is not 6
random_rows = np.random.randint(0,X_six.shape[0],6000)
X_not_six = X_not_six[random_rows]
Y_not_six = Y_not_six[random_rows]

In [4]:
#Appending the data with output as 6 and data with output as not six
X_train = np.append(X_six,X_not_six)
#Reshaping the appended data to appropraite form
X_train = X_train.reshape(X_six.shape[0] + X_not_six.shape[0], 784)

In [ ]:
def build_model():
    
    # this is the size of our encoded representations
    encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats
    
    # this is our input placeholder
    input_img = Input(shape=(784,))
    # "encoded" is the encoded representation of the input
    encoded = Dense(encoding_dim, activation='relu')(input_img)
    # "decoded" is the lossy reconstruction of the input
    decoded = Dense(784, activation='sigmoid')(encoded)
    
    # this model maps an input to its reconstruction
    autoencoder = Model(input=input_img, output=decoded)
    
    # this model maps an input to its encoded representation
    encoder = Model(input=input_img, output=encoded)
    
    # create a placeholder for an encoded (32-dimensional) input
    encoded_input = Input(shape=(encoding_dim,))
    #retrieve the last layer of the autoencoder model
    decoder_layer = autoencoder.layers[-1]
    # create the decoder model
    decoder = Model(input=encoded_input, output=decoder_layer(encoded_input))
    
    autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
    autoencoder.fit(X_train, X_train,
                nb_epoch=50,
                batch_size=256,
                shuffle=True,validation_data=(X_test, X_test))
    
    # encode and decode some digits
    # note that we take them from the *test* set
    encoder.compile(optimizer='adadelta', loss='binary_crossentropy')
    decoder.compile(optimizer='adadelta', loss='binary_crossentropy')
    encoded_imgs = encoder.predict(X_test)
    decoded_imgs = decoder.predict(encoded_imgs)
    
    # use Matplotlib 
    
    n = 10  # how many digits we will display
    plt.figure(figsize=(20, 4))
    for i in range(n):
        # display original
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(X_test[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        
        # display reconstruction
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(decoded_imgs[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

In [ ]:
%timeit -n1 -r1 build_model()

Train on 11918 samples, validate on 6000 samples
Epoch 1/50
11918/11918 [==============================] - 1s - loss: 0.6261 - val_loss: 0.4316
Epoch 2/50
11918/11918 [==============================] - 1s - loss: 0.3130 - val_loss: 0.2993
Epoch 3/50
11918/11918 [==============================] - 1s - loss: 0.2750 - val_loss: 0.2896
Epoch 4/50
11918/11918 [==============================] - 1s - loss: 0.2697 - val_loss: 0.2853
Epoch 5/50
11918/11918 [==============================] - 1s - loss: 0.2669 - val_loss: 0.2835
Epoch 6/50
11918/11918 [==============================] - 1s - loss: 0.2645 - val_loss: 0.2806
Epoch 7/50
11918/11918 [==============================] - 1s - loss: 0.2620 - val_loss: 0.2780
Epoch 8/50
11918/11918 [==============================] - 1s - loss: 0.2590 - val_loss: 0.2741
Epoch 9/50
11918/11918 [==============================] - 1s - loss: 0.2552 - val_loss: 0.2706
Epoch 10/50
11918/11918 [==============================] - 1s - loss: 0.2507 - val_loss: 0.2654
